<h1><center><font color=D00000>Q-learning в средах OpenAI.gym</font></center></h1>

### <font color=black>Евгений Пономарев</font>
#### <font color=green>Сколковский институт науки и технологий</font>

Библиотеки:
* `pip install numpy`
* `pip install tensorflow`
* `pip install gym`
* `pip install gym[atari]`
* `pip install keras`
* `pip install matplotlib`

Документация к openai.gym:
* https://gym.openai.com/docs/

<h2><center><font color=D00000>Обучение с подкреплением aka Reinforcement learning:</font></center></h2>
 
В классической постановке предполагается **Марковский процесс принятия решений (Markov Decision Process)**:

Состоит из:
* Среды
* Агента
    
**MDP** задается через: 
* Множество состояний среды $\mathrm{S} : s_t \in \mathrm{S}$
* Доступные действия $\mathrm{A}: a_t \in \mathrm{A}$
* Модель среды (обычно неизвестна агенту): $s_{t+1} \sim \mathrm{P}_{env}(s_{t+1} | s_t, a_t)$
* Функция награды (обычно неизвестна агенту): $r_{t} \sim \mathrm{R}(r_{t} | a_t, s_t)$
  
Агент совершает действия в соответствии с выработанной политикой:
* $a_t \sim \pi(a_t|s_t)$

<font color=green>Цель агента - максимизировать ожидание кумулятивной награды:
$$R(\pi) = \mathbb{E}_{\pi, P, R} \sum_{t=start}^{end}{r_t} \to \max_{\pi}$$
    </font></center>

<img src="http://pp.userapi.com/c621707/v621707705/23555/2WlxiB6Di3o.jpg" alt="MDP" style="width: 600px;"/>

Основной плюс - можно не размечать данные


# Табличный Q-learning

Сопоставим каждой паре $(s,a)$ (состояние, действие) значение функции ценности действия - **$Q(s,a)$**
$$Q(s_t, a_t) = \mathbb{E}_{\pi, P, R} \sum_{i=0}^{N}{\gamma^ir_{t+i}}$$

Тогда награда за $$R(\pi) = \sum_{t=start}^{end}{r_t} \sim \sum_{t=start}^{end} Q(s_t|a_t)$$


Искомая $Q$ функция:
$$Q^*(s,a) = max_{\pi}Q^{\pi}(s,a)$$

Тогда жадная политика
$$ \pi(a|s) = \mathbb{[}a == \arg\max_a Q(s,a)\mathbb{]} $$
$$ a_t = \arg\max_a Q(s,a) $$
решает исходную задачу

Уравнение Беллмана для оптимальной функции:

$$Q^*(s,a) = \mathbb{E}_{P_{env}}(r_{t+1} + \gamma \max_{a'\in A} Q^*(s_{t+1},a')~~|~~s_t = s, a_t = a)$$




$$Q(s_{t}, a_t) = r_t + \gamma \cdot \max_{a' \in A}Q(s_{t+1}|a')$$



В простейшем случае, когда s~1,10,100; a~1,10 лучше просто выучить, что нужно делать, т.е. задать таблицу: 

| Q(s,a)| $s_1$ | $s_2$  | $s_3$ | ... |
|-----|-----|------|-----|-----|
| $a_1$ | 1   | 0    | 8   | ... |
| $a_2$ | 5   | 4    | 3   | ... |
| $a_3$ | 8   | -10 | 99  | ... |

Оценка $Q(s, a)$ экспоненциальным скользящим средним:


$$ Q(s, a) := Q(s, a) + \alpha \cdot [(r(a,s) + \max_{a'\ \in A} Q(s',a')) - Q(s,a)] $$

$$ Q(s,a) \to r(a,s) + \max Q(s')$$

<h2><center><font color=#D00000>CartPole</font></center></h2>

![cartpole](cartpole_gif.gif)
## Правила

### Вектор наблюдения

Num | Observation | Min | Max
---|---|---|---
0 | Позиция тележки | -2.4 | 2.4
1 | Скорость тележки | -Inf | Inf
2 | Угол наклона палки | ~ -41.8&deg; | ~ 41.8&deg;
3 | Угловая скорость палки | -Inf | Inf

### Действия

Num | Action
--- | ---
0 | Толкнуть влево
1 | Толкнуть вправо

Note: The amount the velocity is reduced or increased is not fixed as it depends on the angle the pole is pointing. This is because the center of gravity of the pole increases the amount of energy needed to move the cart underneath it

### Награда
1 за каждый шаг

### Начальное состояние
Случайные числа ±0.05

### Завершение эпизода
1. Наклон палки = ±20.9°
2. Положение тележки = ±2.4 (центр тележки за пределами экрана)
3. Eлительность эпизода 200 (v.0) или 500 (v.1)

### Условия победы
Продержаться 100 последовательных эпизодов со средним результатом более 195 (v.0) или 475 (v.1)

In [ ]:
import gym
import numpy as np
import random
import pandas
import matplotlib.pyplot as plt


In [ ]:
class QLearn:
    def __init__(self, actions, epsilon, alpha, gamma):
        self.q = {}
        self.epsilon = epsilon  # exploration constant
        self.alpha = alpha      # discount constant
        self.gamma = gamma      # discount factor
        self.actions = actions

    def chooseAction(self, state, return_q=False):
        q = [self.q.get((state, a), 0.) for a in self.actions]
        maxQ = max(q)
        
        # Отвечает за баланс исследование-использование (exploration vs expluatation)
        # С вероятностью epsilon случайное действие, 1-epsilon - argmax(Q)
        if random.random() < self.epsilon:
            minQ = min(q); mag = max(abs(minQ), abs(maxQ))
            # add random values to all the actions, recalculate maxQ
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))]
            maxQ = max(q)

        count = q.count(maxQ)
        # Если максимум Q достигается для двух и более действий - берем любое
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            i = random.choice(best)
        else:
            i = q.index(maxQ)
        
        action = self.actions[i]
        if return_q: # if they want it, give it!
            return action, q
        return action
    
    def learn(self, state1, action1, reward, state2):
        maxqnew = max([self.q.get((state, action), 0.) for a in self.actions])
        new_q = reward + self.gamma*maxqnew

        old_q = self.q.get((state, action), None)
        if old_q is None:
            self.q[(state, action)] = reward
        else:
            self.q[(state, action)] = old_q + self.alpha * (new_q - old_q)

def build_state(features):
    return int("".join(map(lambda feature: str(int(feature)), features)))

def to_bin(value, bins):
    return np.digitize(x=[value], bins=bins)[0]

In [ ]:
env = gym.make('CartPole-v0')
goal_average_steps = 195
max_number_of_steps = 200
# Для создания видео и отправки данных на сайт gym.openai.com
env = gym.wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)

n_bins = 8
n_bins_angle = 10

# Дискретизуем непрерывное пространство состояний в n_bins^2*n_bins_angle^2
cart_position_bins = pandas.cut([-2.4, 2.4], bins=n_bins, retbins=True)[1][1:-1]
pole_angle_bins = pandas.cut([-2, 2], bins=n_bins_angle, retbins=True)[1][1:-1]
cart_velocity_bins = pandas.cut([-1, 1], bins=n_bins, retbins=True)[1][1:-1]
angle_rate_bins = pandas.cut([-3.5, 3.5], bins=n_bins_angle, retbins=True)[1][1:-1]

# Инициализация табличного Q-learning
alpha = 0.5
gamma = 0.9
epsilon = 0.1
qlearn = QLearn(actions=range(env.action_space.n),
                alpha=alpha, gamma=gamma, epsilon=epsilon)

```C
init s
init Q
eps = 0.1
alpha = 0.5
    
for episode = 1,M do:
    с вероятностью eps:
        a = ramdom(A)
    else:
        a = argmax(Q[s,a])
    
    Совершить действие a;
    Получить награду r;
    Получить новое состояние s_new;
    
    Улучшить стратегию:
        Q[s,a] := Q[s,a] + alpha*((r + max(Q[s_new,:])) - Q[s,a])
    
    s := s_new
```

In [ ]:

last_reward = np.ndarray(0)
for i_episode in xrange(30000):
    
    # получим наблюдение
    observation = env.reset()
    # дискретизуем и создадим состояние
    cart_position, pole_angle, cart_velocity, angle_rate_of_change = observation
    state = build_state([to_bin(cart_position, cart_position_bins),
                     to_bin(pole_angle, pole_angle_bins),
                     to_bin(cart_velocity, cart_velocity_bins),
                     to_bin(angle_rate_of_change, angle_rate_bins)])
    cum_reward = 0
    for t in xrange(max_number_of_steps):
        # можно смотреть, как агент учится
#         if i_episode % 300 == 0: 
#             env.render()
        
        # в соответствии с политикой выберем действие
        action = qlearn.chooseAction(state)
        # получим новое наблюдение, награду, метку конца эпизода
        observation, reward, is_done, _ = env.step(action)
        cum_reward += reward
        # дискретизуем и создадим состояние
        cart_position, pole_angle, cart_velocity, angle_rate_of_change = observation
        new_state = build_state([to_bin(cart_position, cart_position_bins),
                         to_bin(pole_angle, pole_angle_bins),
                         to_bin(cart_velocity, cart_velocity_bins),
                         to_bin(angle_rate_of_change, angle_rate_bins)])


        if not(is_done):
            qlearn.learn(state, action, reward, new_state)
            state = new_state
        else:
            # штраф за проигрыш
            reward = -200
            qlearn.learn(state, action, reward, new_state)
            last_reward = np.append(last_reward, cum_reward)
            break
    print("episode: {}, score: {}, 30-mean: {:0.1f}".format(i_episode, t, last_reward[-30:].mean()))
    if (last_reward[-100:].mean() > goal_average_steps):
        print ("Win!")
        break
env.close()
# gym.upload('/tmp/cartpole-experiment-1', algorithm_id='simple Q-learning', api_key='your-key')

In [ ]:
w = 15
plt.plot(last_reward)
plt.plot([last_reward[max(0, i-w):min(len(last_reward), i+w)].mean() for i in range(len(last_reward))])
plt.legend(['score', 'RM-{}'.format(2*w)])
plt.title('table-QN: alpha = {}, gamma = {}, epsilon = {}'.format(alpha, gamma, epsilon))
plt.xlabel('step')
plt.ylabel('score')
plt.show()

# Deep Q-learning


Если количество состояний велико, даже бесконечно ($~R^4$), то все запомнить не получится.

Приходится использовать апроксимацию Q-функции.

Например аппроксимацию нейронной сетью, т.е. $Q(s,a) = Q_{nn}(s,a|\theta)$, где $\theta$-веса сетки

<img src="https://pp.userapi.com/c840129/v840129239/4afe4/R9TwZiy1LoA.jpg" alt="MDP" style="width: 600px;"/>

Есть только история $(s, a, r, s')$, полученная при следовании данной политике.

Получим N таких значений и пересчитаем параметры $\theta$, т.е. веса нейронной сети:

$$Q_{target}(s,a) = r + gamma\cdot \max_{a'} Q(s',a' | \theta)$$
$$Loss(\theta) = \frac{1}{N} \sum_{(s,a,r,s') \in batch} \left[Q_target(s,a) - Q(s,a | \theta)\right]^2$$



In [ ]:
import gym
import random
import numpy as np
from keras.models import Sequential
from keras import optimizers
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2

In [ ]:
class DeepQ:
    def __init__(self, inputs, outputs, mem_size, gamma, learning_rate, epsilon, batch_size):
        """
        Параметры:
            - inputs: размерность входного вектора
            - outputs: размерность выходного вектора (к-во действий)
            - mem_size: размер буфера памяти
            - gamma: параметр затухания награды
            - learning_rate: множитель перед градиентом в SGD
            - batch_size: размер подвыборки для тренировки
        """
        self.actions = range(outputs)
        self.epsilon = epsilon
        self.input_size = inputs
        self.output_size = outputs
        self.memory = []
        self.mem_size = mem_size
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.global_i = 0
        self.model = self.create_model()
        
    def create_model(self):
        model = Sequential()
        hiddenLayers = [30,30]
#         hiddenLayers = [30]
        activationType = 'relu'
        model.add(Dense(hiddenLayers[0], input_shape=(self.input_size,), init='lecun_uniform'))
        model.add(Activation(activationType))

        for index in range(1, len(hiddenLayers)):
            model.add(Dense(hiddenLayers[index], init='lecun_uniform'))
            model.add(Activation(activationType))

#       На выходе числа из R
        model.add(Dense(self.output_size, init='lecun_uniform'))
        model.add(Activation("linear"))
        
#       Оптимизатор, чтобы устремлять Q -> Q_target
        optimizer = optimizers.RMSprop(lr=learning_rate, rho=0.9, epsilon=1e-06)
    
#       Ф-я потерь sum((Q[:]-Q_target[:])**2)
        model.compile(loss="mse", optimizer=optimizer)
        model.summary()
        return model

    def get_Q_values(self, state):
        predicted = self.model.predict(state.reshape(1,-1))
        return predicted[0]
    
    def chooseAction(self, state):
        q = self.get_Q_values(state).tolist()
        maxQ = max(q)
        
        # Отвечает за баланс исследование-использование (exploration vs expluatation)
        # С вероятностью epsilon случайное действие, 1-epsilon - argmax(Q)
        if random.random() < self.epsilon:
            minQ = min(q); mag = max(abs(minQ), abs(maxQ))
            q = [q[i] + random.random() * mag - .5 * mag for i in range(len(self.actions))]
            maxQ = max(q)

        count = q.count(maxQ)
        
        # Если максимум Q достигается для двух и более действий - берем любое
        if count > 1:
            best = [i for i in range(len(self.actions)) if q[i] == maxQ]
            action = random.choice(best)
        else:
            action = q.index(maxQ)
        return action
    
        # Пересчет значения Q(s,a) := r + gamma * max Q(s',a')
    def calc_target(self, Q_values, reward, is_done):
        if is_done:
            return reward
        else : 
            return reward + self.gamma * max(Q_values)
    
    def remember(self, sarsd):
        if len(self.memory) < self.mem_size:
            self.memory.append(sarsd)
        else:
            self.memory[self.global_i%self.mem_size] = sarsd
        self.global_i += 1
        
    def learn(self):
        #выберем случайную подвыборку длины self.batch_size
        batch_indexes = random.sample(range(len(self.memory)), self.batch_size)
        
        #контейнеры  X = вход; Y = ответ
        X_batch = np.zeros((self.batch_size, self.input_size))
        Y_batch = np.zeros((self.batch_size, self.output_size))
        j = 0

        for i in batch_indexes:
        # выбираем из памяти s_t, a_t, r_t, s_{t+1} и метку конца 
            state, action, reward, new_state, is_done = self.memory[i]

            # Q_target(s,a) := r + gamma * max Q(s',a')
            Q_values = self.get_Q_values(state)
            Q_values_new = self.get_Q_values(new_state)
            Q_target = self.calc_target(Q_values_new, reward, is_done)

            # Для всех a, кроме a_t не трогаем, для a_t меняем
            X_batch[j] = np.array([state.copy()])
            Y_sample = Q_values.copy()
            Y_sample[action] = Q_target
            Y_batch[j] = np.array([Y_sample])
            
            
            if is_done:
                X_batch[j] = np.array([new_state.copy()])
                Y_batch[j] = np.array([[reward]*self.output_size])
            j += 1
            
#       Оптимизатор устремляет Q -> Q_target
#       т.е. делает 1 шаг против направления градиента ф-и потерь L = sum((Q[:]-Q_target[:])**2) по параметрам сетки
        self.model.fit(X_batch, Y_batch, batch_size = self.batch_size, nb_epoch=1, verbose = 0)

In [ ]:
# env = gym.make('MsPacman-v0')
env = gym.make('CartPole-v1')
goal_average_steps = 475
goal_number_of_steps = 500

# env = gym.wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)

mem_size = 1024
gamma = 0.99
learning_rate = 0.00525
epsilon = 0.1

batch_size  = 256
learn_step = 256
last_reward = np.ndarray(0)
state = env.reset().ravel()
# Инициализация глубокого Q-learning
qlearn = DeepQ(state.shape[0], env.action_space.n, mem_size, gamma, learning_rate, epsilon, batch_size)

for i_episode in xrange(3000):
    state = env.reset().ravel()
    cum_reward = 0
    for t in xrange(goal_number_of_steps):
#         можно смотреть, как агент учится
        if i_episode % 100 == 0: 
            env.render()

        # в соответствии с политикой выберем действие
        action = qlearn.chooseAction(state)
        
        # получим новое наблюдение, награду, метку конца эпизода
        new_state, reward, is_done, info = env.step(action)
        new_state = new_state.ravel()
        cum_reward += reward
        if not(is_done):
            qlearn.remember((state, action, reward, new_state, is_done))
            state = new_state
        else:
        # штраф за поражение
            reward = -200
            qlearn.remember((state, action, reward, new_state, is_done))
            last_reward = np.append(last_reward, cum_reward)
            break
            
    if qlearn.global_i > learn_step:
#         print(qlearn.global_i)
        qlearn.learn()
    print("episode: {}, score: {}, 30-mean: {:0.1f}".format(i_episode, t, last_reward[-30:].mean()))
    if (last_reward[-100:].mean() > goal_average_steps):
        print ("Win!")
        break

env.close()
# gym.upload('/tmp/cartpole-experiment-1', algorithm_id='vmayoral simple Q-learning', api_key='your-key')

In [ ]:
w = 15
plt.plot(last_reward)
plt.plot([last_reward[max(0, i-w):min(len(last_reward), i+w)].mean() for i in range(len(last_reward))])
plt.legend(['score', 'RM-{}'.format(2*w)])
plt.title('DQN')
plt.xlabel('step')
plt.xlabel('score')
plt.show()

## Домашнее задание

0. Проследить зависимость результата от epsilon, построить графики для epsilon = [0.01, 0.07, 0.2, 0.5];
0. Проследить зависимость результата от learning_rate, построить графики для разных learning_rate;
0. Динамически менять epsilon и learning_rate, обучиться быстрее и стабильнее
0. Усложнить модель, добавить слои, поменять число нейронов, ф-ию активации;
0. $^*$ Набрать 1000 очков в [Ms.Packman](https://gym.openai.com/envs/MsPacman-v0/) в OpenAI.gym. Скорее всего понадобится добавить пару сверточных слоев

За иллюстрации отдельное спасибо А.Гринчуку, см. [его семинар](rl_advantages.pdf)